# Knapsack Problem Experiments
In the binary knapsack problem you are given:
- a set of objects o1, o2, ..., ok
- their values v1, v2, ..., vk
- their weights w1, w2, ..., wk
The goal is to maximize the total value of the selected objects, subject to a weight constraint w

In [1]:
import numpy as np

## Representation and Toy Problem Generation

In [2]:
class KnapsackProblem:
    num_objects = None
    values = None
    weights = None
    capacity = None
    
    def __init__(self, num_objects: int) -> None:
        self.num_objects = num_objects
        self.values = 2 ** np.random.rand(num_objects)
        self.weights = 2 ** np.random.rand(num_objects)
        self.capacity = 0.25 * self.weights.sum()
            
    def __str__(self):
        return f'Knapsack Problem Object with N={self.num_objects}\n\n' \
            + f'Values: {self.values}\n\n' \
            + f'Weights: {self.weights}\n\n' \
            + f'Capacity: {self.capacity}\n'

In [3]:
kp = KnapsackProblem(10)
print(kp)

Knapsack Problem Object with N=10

Values: [1.66814767 1.55758968 1.00276745 1.79895612 1.0621401  1.40455682
 1.53750476 1.31605103 1.31107832 1.0670016 ]

Weights: [1.94376776 1.99184416 1.90122895 1.27313338 1.02232951 1.99047892
 1.03899075 1.1623158  1.02209891 1.14427486]

Capacity: 3.622615749660481



## Solutions Representation
1) List of integers
2) A permutation of K numbers

In [4]:
class Individual:
    order = None
    alpha = None # probability of mutation
    def __init__(self, kp: KnapsackProblem, alpha: float=0.05) -> None:
        self.order = np.random.permutation(kp.num_objects)
        self.alpha = alpha
        
    def __str__(self) -> str:
        return f'{self.order}'

In [5]:
def fitness(kp: KnapsackProblem,
            individual: Individual) -> float:
    remaining_weight = kp.capacity
    total_value = 0

    for idx in individual.order:
        if kp.weights[idx] < remaining_weight:
            remaining_weight -= kp.weights[idx]
            total_value += kp.values[idx]

    return total_value

In [6]:
def in_knapsack(kp: KnapsackProblem,
               individual: Individual) -> float:
    remaining_weight = kp.capacity
    kpi = set()
    
    for idx in individual.order:
        if kp.weights[idx] < remaining_weight:
            remaining_weight -= kp.weights[idx]
            kpi.add(idx)

    return kpi

In [7]:
kp = KnapsackProblem(10)
ind = Individual(kp)
print(kp)
print(f'fitness of individual {ind}: {fitness(kp, ind):.4f}')

Knapsack Problem Object with N=10

Values: [1.96352026 1.27077357 1.58362964 1.29219692 1.87803282 1.59021832
 1.18337536 1.44794622 1.69147336 1.63832435]

Weights: [1.53838339 1.1578858  1.38901387 1.23472527 1.17644583 1.85385408
 1.09464916 1.15818984 1.15732272 1.65442933]

Capacity: 3.35372482216783

fitness of individual [1 7 9 8 3 6 2 4 5 0]: 2.7187


## Evolutionary Methods

In [8]:
def mutation(ind: Individual) -> Individual:
    if np.random.rand() > ind.alpha:
        i1 = np.random.randint(10)
        i2 = np.random.randint(10)
        ind.order[i1], ind.order[i2] = ind.order[i2], ind.order[i1]
        
    return ind


def recombination(kp: KnapsackProblem,
                  p1: Individual,
                  p2: Individual
                 ) -> Individual:
    
    set1 = in_knapsack(kp, p1)
    set2 = in_knapsack(kp, p2)
    
    # copy intersection to offspring
    offspring = set1.intersection(set2)
    
    # copy rest with probability of 50%
    for ind in set1.symmetric_difference(set2):
        if np.random.rand() <= 0.5:
            offspring.add(ind)
    
    order = list(range(kp.num_objects))
    i = 0
    for obj in offspring:
        order[i] = obj
        i += 1
    
    rem = set(range(kp.num_objects)).difference(offspring)
    for obj in rem:
        order[i] = obj
        i += 1
        
    # randomly permute the elements
    order[:len(offspring)] = [
        order[idx] for idx in np.random.permutation(
        range(len(offspring)))]
    
    order[len(offspring):] = [
        order[idx] for idx in np.random.permutation(
        range(len(offspring), kp.num_objects))]

    beta = 2 * np.random.rand() - 0.5 # between -1.5 and 1.5
    alpha = p1.alpha + beta * (p2.alpha - p1.alpha)
    
    ind = Individual(kp, alpha)
    ind.order = order
    
    return ind
    

def selection(kp: KnapsackProblem, population: list, k: int) -> Individual:
    indices = np.random.choice(range(len(population)), 5)
    selected = [population[idx] for idx in indices]
    fitnesses = [fitness(kp, s) for s in selected]
    max_idx = np.argmax(fitnesses)
    return selected[max_idx]

def elimination(kp: KnapsackProblem, population: list, offspring: list, lambda_: int) -> list:
    combined = population + offspring
    combined = sorted(combined, key=lambda x: fitness(kp, x), reverse=True)
    
    return combined[:lambda_]

## Main Evolutionary Algorithm

In [20]:
def initialize(kp: KnapsackProblem, lambda_: int) -> list:
    return [Individual(kp) for _ in range(lambda_)]


def evolutionaryAlgorithm(kp: KnapsackProblem,
                          lambda_: int,
                          mu: int,
                          num_iterations: int,
                          k: int):

    # initialize population
    population = initialize(kp, lambda_)
    
    # heuristic solution
    heur_order = np.argsort(kp.values/kp.weights)[::-1]
    heur_solution = Individual(kp)
    heur_solution.order = heur_order
    print(f'Heuristic best solution fitness = {fitness(kp, heur_solution)}')
    
    # intialize offspring list
    offspring = [None] * mu
    
    # run evolutionary algorithm
    for ii in range(num_iterations):
        for jj in range(mu):
            
            # select parents
            parent1 = selection(kp, population, k=k)
            parent2 = selection(kp, population, k=k)
            
            # generate offspring based on parents
            offspring[jj] = recombination(kp, parent1, parent2)
            
            # mutate offspring
            offsprting[jj] = mutation(offspring[jj])
        
        # mutation on original population
        population = [mutation(ind) for ind in population]
        
        # elimination
        population = elimination(kp, population, offspring, lambda_=lambda_)
        
        # calculate fitnesses
        fitnesses = [fitness(kp, ind) for ind in population]
        
        print(f'Mean fitness: {sum(fitnesses)/len(fitnesses)}, ', end='')
        print(f'Best fitness: {max(fitnesses)}')
        

In [22]:
num_objects = 50
lambda_ = 100  # population size
mu = 100  # offspring size
k = 5
num_iterations = 30

kp = KnapsackProblem(num_objects)

evolutionaryAlgorithm(kp, lambda_, mu, num_iterations, k)

Heuristic best solution fitness = 25.69674747268049
Mean fitness: 19.793020805341182, Best fitness: 22.67256048072896
Mean fitness: 21.00878940665085, Best fitness: 22.954834625194863
Mean fitness: 22.14406177569351, Best fitness: 23.74507546004563
Mean fitness: 22.968264245926026, Best fitness: 23.830746459789463
Mean fitness: 23.432633379169637, Best fitness: 24.22405828641433
Mean fitness: 23.908268370565327, Best fitness: 24.939669528992514
Mean fitness: 24.245335690381715, Best fitness: 25.120633724798658
Mean fitness: 24.530868114563365, Best fitness: 25.345532889984447
Mean fitness: 24.77085904684084, Best fitness: 25.936149869670775
Mean fitness: 24.990223042571493, Best fitness: 25.936149869670775
Mean fitness: 25.224363578184324, Best fitness: 25.936149869670775
Mean fitness: 25.53365998319878, Best fitness: 25.936149869670775
Mean fitness: 25.880117799429822, Best fitness: 25.95606509867735
Mean fitness: 25.93674732654097, Best fitness: 25.956065098677353
Mean fitness: 25.93

### TODO
Improve methods to consistently outperform the heuristic for problems of size 250-500.